In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sb

# ETL

In [ ]:
df=pd.read_csv('/Users/ruhidmirzayev/Palette/Data/rm-yields-data.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['RM']=df['RM'].astype('str')

## Univariate Analysis

# Year

In [ ]:
df['Year'].nunique()

In [ ]:
len(df['Year'].unique()) - (2022-1938+1)

In [ ]:
df['Year'].unique()

### RM

In [ ]:
df['RM'].unique()

In [ ]:
df['RM'].nunique()

## Missing Values

In [ ]:
df.isna().sum().sort_values().plot(kind='barh')
plt.title('Examinig Missing Values')
plt.xlabel('Number of Missing Values')
plt.ylabel('Crops')
plt.axvline(len(df), linestyle = '--', color='red')
plt.axvline(len(df)/2, linestyle = '--', color='red')
plt.axvline(len(df)*0.25, linestyle = '--', color='red')
plt.show()

### Descriptive Statistics

In [ ]:
df.describe().drop(['Year', 'RM'], axis=1).T['mean'].sort_values().plot(kind='barh')

In [ ]:
bushels=['Winter Wheat', 'Canola', 'Spring Wheat',
       'Durum', 'Oats', 'Lentils', 'Peas', 'Barley', 'Fall Rye', 'Spring Rye', 'Tame Hay', 'Flax']

In [ ]:
df['Lentils']=df['Lentils']/60 # 1 bushel 60 pounds

### Diplicates

In [ ]:
df.duplicated().sum()

In [ ]:
df.shape

### Data Shape

In [ ]:
print('Supposed # of rows', 85*299 - len(df), 'but needs to be 0')

In [ ]:
df.groupby('RM').count()['Year']\
    .sort_values()[:6].sum() - 6*85

In [ ]:
df.groupby('RM').count()['Year']\
    .sort_values()[:6]

### Outliers

In [ ]:
df[bushels].boxplot(figsize=(20,12))

In [ ]:
df[bushels].hist(figsize=(20,12))
plt.show()

In [ ]:
df['Barley'].sort_values()[:75]

In [ ]:
df[df['Spring Wheat'] > 100]

### Correlation

In [ ]:
plt.figure(figsize=(20,12))
sb.heatmap(df.corr(), annot=True, cmap='Greens')
plt.show()

In [ ]:
df.set_index('Year')[bushels]\
        .groupby('Year')\
            .mean()['Canola']\
            .plot(kind='bar', figsize=(20,12))

In [ ]:
df.set_index('Year')[bushels]\
        .groupby('Year')

# GIS Analysis

In [ ]:
gdf=gpd.read_file('/Users/ruhidmirzayev/Palette/Data/RuralMunicipality/RuralMunicipality.shp')

In [ ]:
gdf.info()

In [ ]:
gdf['RMNO']=gdf['RMNO'].astype('str')

### GIS viz by RM

In [ ]:
df.groupby('RM').mean() 

In [ ]:
gdf['RMNO']=gdf['RMNO'].astype('int').astype('string')

In [ ]:
gdf.info()

In [ ]:
25126 - 25312

In [ ]:
pd.merge(df, gdf.rename(columns={'RMNO':'RM'}), on='RM')

In [ ]:
gdf

In [ ]:
df.RM.unique()

In [ ]:
gdf.RMNO.sort_values().unique()

In [ ]:
df.groupby('RM').mean()

In [ ]:
gpd.GeoDataFrame(pd.merge(df, gdf.rename(columns={'RMNO':'RM'}), on='RM')).plot('Canola', legend=True)

In [ ]:
gpd.GeoDataFrame(pd.merge(df.groupby('RM').mean(), gdf.rename(columns={'RMNO':'RM'}), on='RM')).plot('Canola', legend=True)

In [ ]:
gpd.GeoDataFrame(pd.merge(df.loc[df['RM']!='521'].groupby('RM').mean(), gdf.rename(columns={'RMNO':'RM'}), on='RM'))\
    .explore('Canola', legend=True, cmap='Greens', tiles='Stamen Toner',tooltip=['RM','Canola'])

In [ ]:
gpd.GeoDataFrame(pd.merge(df.loc[df['RM']!='521'].groupby('RM').mean(), gdf.rename(columns={'RMNO':'RM'}), on='RM'))\
    .explore(column='Spring Wheat', legend=True, cmap='Greens', tiles='Stamen Toner',tooltip=['RM','Spring Wheat'])

In [ ]:
gpd.GeoDataFrame(pd.merge(df.loc[(df['RM']!='521') & (df['Year']>2012)].groupby('RM').mean(), gdf.rename(columns={'RMNO':'RM'}), on='RM'))\
    .explore(column='Spring Wheat', legend=True, cmap='Greens', tiles='Stamen Toner',tooltip=['RM','Spring Wheat'])

In [ ]:
gpd.GeoDataFrame(pd.merge(df.loc[(df['RM']!='521') & (df['Year']>2012)].groupby('RM').mean(), gdf.rename(columns={'RMNO':'RM'}), on='RM'))\
    .to_file('/Users/ruhidmirzayev/Palette/Data/Average_RM_Yield.geojson', driver='GeoJSON')